In [1]:
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm, trange
import scipy.io as sio

In [2]:
mat_path = 'mpii_human_pose_v1_u12_2/mpii_human_pose_v1_u12_1.mat'
data = sio.loadmat(mat_path, struct_as_record=False)['RELEASE']

In [3]:
annolist = data[0, 0].__dict__['annolist']
img_tra = data[0, 0].__dict__['img_train']
person = data[0, 0].__dict__['single_person']
act = data[0, 0].__dict__['act']
print(annolist.shape, img_tra.shape, person.shape,act.shape)

(1, 24987) (1, 24987) (24987, 1) (24987, 1)


In [4]:
print(person[123, 0][0][0]) # number_of_person

1


In [5]:
COLS = ['filename', 'xmin', 'ymin', 'xmax', 'ymax', 'scale','activity','category']
print(len(COLS))

8


In [6]:
POINT_COLS = ['right_ankle','right_knee','right_hip', 'left_hip', 'left_knee', 'left_ankle','pelvis','thorax','upper_neck', 'head_top', 'right_wrist','right_elbow', 'right_shoulder','left_shoulder','left_elbow', 'left_wrist']

POINT_COLS_x = [item+'_x' for item in POINT_COLS]
POINT_COLS_y = [item+'_y' for item in POINT_COLS]
POINT_COLS_invis = [item+'_invis' for item in POINT_COLS]

POINT_COLS_ = np.append(POINT_COLS_x, [POINT_COLS_y, POINT_COLS_invis])
print(len(POINT_COLS), len(POINT_COLS_))

COLS_ = np.append(COLS, POINT_COLS_)
print(len(COLS_))

16 48
56


In [7]:
dict_item = {key: [] for key in COLS_}
print(dict_item)

{'filename': [], 'xmin': [], 'ymin': [], 'xmax': [], 'ymax': [], 'scale': [], 'activity': [], 'category': [], 'right_ankle_x': [], 'right_knee_x': [], 'right_hip_x': [], 'left_hip_x': [], 'left_knee_x': [], 'left_ankle_x': [], 'pelvis_x': [], 'thorax_x': [], 'upper_neck_x': [], 'head_top_x': [], 'right_wrist_x': [], 'right_elbow_x': [], 'right_shoulder_x': [], 'left_shoulder_x': [], 'left_elbow_x': [], 'left_wrist_x': [], 'right_ankle_y': [], 'right_knee_y': [], 'right_hip_y': [], 'left_hip_y': [], 'left_knee_y': [], 'left_ankle_y': [], 'pelvis_y': [], 'thorax_y': [], 'upper_neck_y': [], 'head_top_y': [], 'right_wrist_y': [], 'right_elbow_y': [], 'right_shoulder_y': [], 'left_shoulder_y': [], 'left_elbow_y': [], 'left_wrist_y': [], 'right_ankle_invis': [], 'right_knee_invis': [], 'right_hip_invis': [], 'left_hip_invis': [], 'left_knee_invis': [], 'left_ankle_invis': [], 'pelvis_invis': [], 'thorax_invis': [], 'upper_neck_invis': [], 'head_top_invis': [], 'right_wrist_invis': [], 'right

In [8]:
                                # [0, Image ID]
print(1, annolist[0, 24984].__dict__['annorect'][0, 0].__dict__['annopoints'][0, 0].__dict__['point'][0, 0].__dict__['id'])

# every person
print(2,annolist[0, 10].__dict__['annorect'].shape)

                                            # [0, num_people]
print(3,annolist[0, 6].__dict__['annorect'][0, 0].__dict__['x1'])
print(4,annolist[0, 6].__dict__['annorect'][0, 0].__dict__['y1'])
print(5,annolist[0, 6].__dict__['annorect'][0, 0].__dict__['x2'])
print(6,annolist[0, 6].__dict__['annorect'][0, 1].__dict__['y2'])

print(7,annolist[0, 9].__dict__['image'][0, 0].__dict__['name'])

print(8,annolist[0, 24984].annorect[0, 0].__dict__['annopoints'][0,0].__dict__['point'][0 ,15].x)

# ['cat_name', 'act_name', 'act_id']
print(9, act[4, 0].__dict__)

1 [[6]]
2 (1, 1)
3 [[914]]
4 [[79]]
5 [[1031]]
6 [[221]]
7 ['070807258.jpg']
8 [[268]]
9 {'_fieldnames': ['cat_name', 'act_name', 'act_id'], 'cat_name': array(['sports'], dtype='<U6'), 'act_name': array(['curling'], dtype='<U7'), 'act_id': array([[1]], dtype=uint8)}


In [9]:
keypoint_ = [6, 7, 8, 9, 0, 1, 2, 3, 4, 5, 10, 11, 12, 13, 14, 15]

In [10]:
for idx in trange(annolist.shape[1]):
    if img_tra[0, idx] == 0: continue # test-image

    annorect = annolist[0, idx].annorect
    if annorect.shape[0]==0: continue
    num_person = annorect.shape[1]

    for i_person in range(num_person):
        annorect_person = annorect[0, i_person]

        if 'annopoints' not in annorect_person._fieldnames: continue
        annopoints = annorect_person.annopoints
        if annopoints.shape[0]== 0: continue
        point = annopoints[0, 0].point

        filename = annolist[0, idx].image[0, 0].name[0]
        dict_item['filename'].append(filename)

        # BBOX
        xmin = annorect_person.x1[0][0]
        dict_item['xmin'].append(xmin)
        ymin = annorect_person.y1[0][0]
        dict_item['ymin'].append(ymin)
        xmax = annorect_person.x2[0][0]
        dict_item['xmax'].append(xmax) 
        ymax = annorect_person.y2[0][0]
        dict_item['ymax'].append(ymax)

        scale = annorect_person.scale[0][0]
        dict_item['scale'].append(scale)

        if len(act[idx, 0].act_name) != 0: 
            activity = act[idx, 0].act_name[0]
            category = act[idx, 0].cat_name[0]
        else:
            activity = None
            category = None

        dict_item['activity'].append(activity)        
        dict_item['category'].append(category)
        
        for idx_keypoint, i_keypoint in enumerate(keypoint_):
            if idx_keypoint >= point.shape[1]:
                joint_name = POINT_COLS[i_keypoint]
                dict_item[joint_name+'_x'].append(0)
                dict_item[joint_name+'_y'].append(0)
                dict_item[joint_name+'_invis'].append(0)
                continue

            point_i=  point[0, idx_keypoint]
            point_id = point_i.id[0][0]

            if i_keypoint != point_id:
                joint_name = POINT_COLS[i_keypoint]
                dict_item[joint_name+'_x'].append(0)
                dict_item[joint_name+'_y'].append(0)
                dict_item[joint_name+'_invis'].append(0)
                continue

            point_x = point_i.x[0][0]
            point_y = point_i.y[0][0]
            
            
            if 'is_visible' not in point_i._fieldnames:
                point_isvis = 0
            elif len(point_i.is_visible) != 0:
                point_isvis = point_i.is_visible[0][0]
            else : point_isvis = 0
            
            joint_name = POINT_COLS[point_id]
            dict_item[joint_name+'_x'].append(point_x)
            dict_item[joint_name+'_y'].append(point_y)
            dict_item[joint_name+'_invis'].append(point_isvis)

  0%|          | 0/24987 [00:00<?, ?it/s]

In [11]:
for key, value in dict_item.items():
    print(key, len(value))

filename 28883
xmin 28883
ymin 28883
xmax 28883
ymax 28883
scale 28883
activity 28883
category 28883
right_ankle_x 28883
right_knee_x 28883
right_hip_x 28883
left_hip_x 28883
left_knee_x 28883
left_ankle_x 28883
pelvis_x 28883
thorax_x 28883
upper_neck_x 28883
head_top_x 28883
right_wrist_x 28883
right_elbow_x 28883
right_shoulder_x 28883
left_shoulder_x 28883
left_elbow_x 28883
left_wrist_x 28883
right_ankle_y 28883
right_knee_y 28883
right_hip_y 28883
left_hip_y 28883
left_knee_y 28883
left_ankle_y 28883
pelvis_y 28883
thorax_y 28883
upper_neck_y 28883
head_top_y 28883
right_wrist_y 28883
right_elbow_y 28883
right_shoulder_y 28883
left_shoulder_y 28883
left_elbow_y 28883
left_wrist_y 28883
right_ankle_invis 28883
right_knee_invis 28883
right_hip_invis 28883
left_hip_invis 28883
left_knee_invis 28883
left_ankle_invis 28883
pelvis_invis 28883
thorax_invis 28883
upper_neck_invis 28883
head_top_invis 28883
right_wrist_invis 28883
right_elbow_invis 28883
right_shoulder_invis 28883
left_sh

In [12]:
df = pd.DataFrame(dict_item)

In [13]:
df.head()

,filename,xmin,ymin,xmax,ymax,scale,activity,category,right_ankle_x,right_knee_x,...,pelvis_invis,thorax_invis,upper_neck_invis,head_top_invis,right_wrist_invis,right_elbow_invis,right_shoulder_invis,left_shoulder_invis,left_elbow_invis,left_wrist_invis
0,015601864.jpg,627,100,706,198,3.021046,curling,sports,620,616,...,0,1,0,0,1,1,1,1,1,1
1,015601864.jpg,841,145,902,228,2.472117,curling,sports,895,910,...,0,0,0,0,1,1,0,1,1,1
2,015599452.jpg,607,70,752,255,5.641276,curling,sports,0,0,...,0,0,0,0,0,0,0,0,0,0
3,015599452.jpg,903,73,1070,263,6.071051,curling,sports,0,0,...,0,0,0,0,0,0,0,0,0,0
4,015599452.jpg,27,36,186,214,5.728162,curling,sports,0,0,...,0,0,0,0,0,0,0,0,0,0


In [15]:
df.to_csv('train_mpii_datasets.csv', index=False)